## lightGBMをGPUで使用する準備

In [ ]:
# !pip uninstall -y lightgbm
# !apt-get install -y libboost-all-dev
# !git clone --recursive https://github.com/Microsoft/LightGBM

In [ ]:
# %%bash
# cd LightGBM
# rm -r build
# mkdir build
# cd build
# cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
# make -j$(nproc)

In [ ]:
# !cd LightGBM/python-package/;python setup.py install --precompile

In [ ]:
# !mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
# !rm -r LightGBM

## ライブラリのインポート

In [ ]:
import gc

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import lightgbm as lgb
from sklearn.model_selection import train_test_split

import shap

## データの取得、確認
kaggleのサイトにある、データの説明は下記の通り
```
■データフィールド
Elevation - メートル単位の標高
Aspect - アスペクト（方位角）です。
Slope - 傾斜の度合いを表す。
Horizontal_Distance_To_Hydrology - 最も近い地表水域の特徴までの距離（Horz
Vertical_Distance_To_Hydrology - 最も近い地表水域までの垂直方向の距離。
Horizontal_Distance_To_Roadways - 最も近い道路までの距離（Horz）。
Hillshade_9am (0 to 255 index) - 夏至の午前9時のヒルシェード指数
Hillshade_Noon（0から255までの値） - 夏至の正午における日覆い指数
Hillshade_3pm（0〜255） - 夏至の午後3時の日陰率
Horizontal_Distance_To_Fire_Points - 最も近い山火事の発火点までの距離。
Wilderness_Area (4つのバイナリ列、0 = 存在しない、1 = 存在する) - 原生地域の指定。
Soil_Type (40個のバイナリ列、0 = 無し、1 = 有り) - 土壌タイプの指定
Cover_Type (7タイプ、1～7の整数) - 森林被覆タイプの指定

原生地域は

1 - ラワ・ウィルダネス・エリア
2 - ネオタ原生地域（Neota Wilderness Area
3 - コマンチ・ピーク・ウィルダネス・エリア
4 - Cache la Poudre Wilderness Area（キャッシュ・ラ・プードル原生地域

土壌の種類は以下の通りです。

1 カテドラル・ファミリー - 岩石露頭の複合体、非常に石が多い。
2 バネット - ラタケ族の複合体、非常に石が多い。
3 Haploborolis - 岩石露頭の複合体、こわれやすい。
4 ラタケ族-岩石露頭の複合体、擦れやすい。
5 バネットファミリー - 岩石露頭の複合体、擦り切れやすい。
6 バネット-ウエットモアファミリー - 岩石露頭複合体、石状。
7 ゴシック族。
8 スーパーバイザー-リンバーファミリーの複合体。
9 トラウトビル・ファミリー 非常に石が多い。
10 ブルワーク-キャタマウント・ファミリー - 岩石露頭の複合体、砕けやすい。
11 ブルワーク-キャタマウント・ファミリー-岩地複合体、擦り切れやすい。
12 レゴー家 - 岩地複合体、石ころだらけ。
13 カタマウント家-岩地-ブルウォーク家の複合体、擦れている。
14 パクチー・アルギボリス - アクオリス複合体。
15 USFSのSoil and ELU Surveyでは特定されていない。
16 Cryaquolis - クライオボローリス複合体。
17 ゲートビューファミリー - クライアクオリスコンプレックス。
18 ロガートファミリー、非常に石が多い。
19 典型的なクライアクオリス - ボロヘムスコンプレックス。
20 タイピック・クライアクォーツ - タイピック・クライアクォーツ複合体。
21 典型的なクリヤコルス - リーカン科、基底膜までの複合体。
22 リーカン科、岩層、非常に岩石が多い。
23 リーカン・ファミリー、基層まで-典型的なクライアクォールズの複合体。
24 リーカン・ファミリー、極めて石の多いもの。
25 リーカン・ファミリー、暖かい、極めて石の多い。
26 グラナイル-カタマウント族複合体、非常に石が多い。
27 リーカン・ファミリー、暖かい - 岩石露頭の複合体、非常に石が多い。
28 リーカン・ファミリー - 岩石露頭の複合体、非常に石が多い。
29 コモ-レゴー家の複合体、極めて石が多い。
30 コモ・ファミリー-岩地-レゴー・ファミリー複合体、極めて石が多い。
31 リーカン-カタマウント・ファミリーの複合体、極めて石が多い。
32 カタマウント・ファミリー-岩場-リーカン・ファミリーの複合体、極めて石が多い。
33 リーカン-キャタマウント・ファミリー-岩石露頭複合体、極めて石が多い。
34 クライオルセント-岩地の複合体、極めて石が多い。
35 クライアンブレプト族-岩石露頭-クライアンブレプト族の複合体。
36 ブロス・ファミリー-岩地-クライアンブレプト複合体、極めて石が多い。
37 岩場の露頭 - クライアンブレプト - クライアンブレプト複合体、極めて石が多い。
38 リーカン家-モラン家-クライアクオルツ複合体、極めて石が多い。
39 モラン族-クライオルセント族-リーカン族の複合体、極めて石が多い。
40 モラン族-クライオセント族-ロックランド族の複合体、極めて石が多い。
```

In [ ]:
train_df = pd.read_csv("../input/tabular-playground-series-dec-2021/train.csv")
train_df.head()

In [ ]:
test_df = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")
test_df.head()

In [ ]:
sample_df = pd.read_csv("../input/tabular-playground-series-dec-2021/sample_submission.csv")
sample_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.describe()

In [ ]:
test_df.info()

In [ ]:
test_df.describe()

## メモリ削減

In [ ]:
def reduce_memory_usage(df):
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    return df

In [ ]:
train_df = reduce_memory_usage(train_df)
test_df = reduce_memory_usage(test_df)

In [ ]:
train_df.info()

In [ ]:
test_df.info()

## データの可視化
### seabornを使って可視化
棒グラフはcountplot、ヒストグラムはhistplotで描画することが可能。<br>
連続値かどうかの判断を、各カラムのユニークな値の数が、スタージェスの公式で求めたビン数より多いかどうかで判断し、それに応じてグラフの種類を出し分け。<br>
plt.figureで名前を指定して描画域を分けることにより、項目名のループでまとめてグラフを描画している。<br>
また、FaceTGridを使うことで、Cover_Typeごとの分布も確認。

### スタージェスの公式
ヒストグラムのビンの数を決める際に用いられる公式。<br>
https://best-biostatistics.com/excel/sturges.html

In [ ]:
def sturges_rule(n):
    return round(1 + math.log2(n))

In [ ]:
for i,colname in enumerate(train_df.columns):
    plt.figure("train" + str(i))
    if len(train_df[colname].unique()) <= sturges_rule(len(train_df)):
        sns.countplot(x=colname,  data=train_df)
    else:
        sns.histplot(train_df[colname], kde=False, bins=sturges_rule(len(train_df))) 

In [ ]:
for i,colname in enumerate(test_df.columns):
    plt.figure("test" + str(i))
    if len(test_df[colname].unique()) <= sturges_rule(len(train_df)):
        sns.countplot(x=colname,  data=test_df)
    else:
        sns.histplot(test_df[colname], kde=False, bins=sturges_rule(len(train_df))) 

In [ ]:
for i,colname in enumerate(train_df.columns):
    plt.figure("hue" + str(i))
    if len(train_df[colname].unique()) <= sturges_rule(len(train_df)):
        grid = sns.FacetGrid(train_df, col='Cover_Type', hue='Cover_Type', col_wrap=3, height=5)
        grid.map(sns.countplot, colname)
        plt.show()
    else:
        grid = sns.FacetGrid(train_df, col='Cover_Type', hue='Cover_Type', col_wrap=3, height=5)
        grid.map(sns.histplot, colname, bins=sturges_rule(len(train_df)), kde=True)
        plt.show()

## Cover_Typeとの相関を確認

In [ ]:
corr = train_df.corr()
plt.figure("corr", figsize = [25,20])
sns.heatmap(corr, square=True, annot=True)

## 入力データ作成

In [ ]:
feature_df = train_df.copy()
feature_df = feature_df.drop("Id", axis=1)
feature_df = feature_df.drop("Cover_Type", axis=1)

In [ ]:
label_df = pd.DataFrame(train_df["Cover_Type"])
label_df = label_df - 1 #light gbmは0からのクラスにする必要がある。

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
  feature_df, label_df, test_size=0.3, random_state=0
)

## ベースラインモデルの作成
kaggleでもよく使われるlight-gbmを使用する。<br>
Cover_Typeが1-7まであるので、multi classを使う。<br>
学習時間の節約のため、アーリーストップを指定している。

In [ ]:
# params = {
# #    'device_type': 'gpu',
#     'boosting_type': 'gbdt',
#     'objective': 'multiclass',
#     'metric': 'multi_logloss',
#     'num_class': 7, 
#     'random_state': 0,
#     'verbose' : -1,
#     'lambda_l1': 2, 
#     'lambda_l2': 9, 
#     'num_leaves': 93, 
#     'min_child_samples': 84
# }

# lgb_train = lgb.Dataset(X_train, y_train)
# lgb_val = lgb.Dataset(X_test, y_test, reference=lgb_train)

# gbm = lgb.train(params,
#                 lgb_train,
#                 valid_sets=[lgb_train, lgb_val],
#                 num_boost_round = 1000,
#                 early_stopping_rounds=100,
#                 verbose_eval=100)

## 特徴量の重要度の確認
* 木の分岐回数が多かった特徴量(importance_type = 'split')
* gini係数の改善に役立った特徴量(importance_type = 'gain')

In [ ]:
# lgb.plot_importance(gbm, figsize=(10,10), max_num_features=50, importance_type='split')

In [ ]:
# lgb.plot_importance(gbm, figsize=(10,10), max_num_features=50, importance_type='gain')

## SHAP値の確認

In [ ]:
# #notebook内でJavascriptを動かすためのおまじない
# shap.initjs()

# """
# shap.TreeExplainer:決定木用(XGBoost、lightBGM等含む)
# shap.LinearExplainer :線形モデル用
# shap.DeepExplainer :Deeplearning用
# """
# #TreeExplainerは、決定木系のモデルのSHAP値を取得するもの。
# explainer = shap.TreeExplainer(model=gbm)
# print(explainer.expected_value)

In [ ]:
# X_test_shap = X_test.sample(frac=0.05)
# class_names = [0, 1, 2, 3, 4, 5, 6]
# shap_values = explainer.shap_values(X=X_test_shap)
# shap.summary_plot(shap_values, X_test_shap, plot_type="bar", 
#                   class_names= class_names, feature_names = X_test_shap.columns)

## モデルのチューニング
モデルのチューニングとして、下記２つを実施。
### K-Fold交差検証
https://axa.biopapyrus.jp/machine-learning/model-evaluation/k-fold-cross-validation.html
### ハイパーパラメータチューニング
Optunaというベイズ最適化用のライブラリを使用して実施。<br>
[ベイズ最適化](https://book.mynavi.jp/manatee/detail/id=59393)<br>
[Oputuna](https://optuna.org/)

In [ ]:
from sklearn.model_selection import StratifiedKFold
import optuna
from sklearn.metrics import accuracy_score

## ハイパーパラメータチューニングのみ

In [ ]:
def opt_LGBM(trial):
    lambda_l1 = trial.suggest_float('lambda_l1', 0.1, 9.9)
    lambda_l2 = trial.suggest_float('lambda_l2', 0.1, 9.9)
    num_leaves = trial.suggest_int('num_leaves', 10, 100)
    min_child_samples = trial.suggest_int('min_child_samples', 10, 100)
    num_boost_round = trial.suggest_int('num_boost_round', 1000, 5000, 1000)
    params = {
#        'device_type': 'gpu',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'num_class': 7, 
        'random_state': 0,
        'verbose' : -1,
        'lambda_l1' : lambda_l1,
        'lambda_l2' : lambda_l2,
        'num_leaves' : num_leaves,
        'min_child_samples' : min_child_samples,
    }

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_test, y_test, reference=lgb_train)

    model = lgb.train(params,
                    lgb_train,
                    valid_sets=[lgb_train, lgb_val],
                    num_boost_round = num_boost_round,
                    early_stopping_rounds=100,
                    verbose_eval=100)

    y_valid_pred = model.predict(X_test)
    score = accuracy_score(y_test, np.argmax(y_valid_pred, axis=1))
    print(f'ACC: {score}')

    return score

## K-Fold
時間がとてつもなくかかるので一旦やらない

In [ ]:
# def opt_LGBM(trial):
#     lambda_l1 = trial.suggest_int('lambda_l1', 0.1, 9.9, 0.1)
#     lambda_l2 = trial.suggest_int('lambda_l2', 0.1, 9.9, 0.1)
#     num_leaves = trial.suggest_int('num_leaves', 1, 100)
#     min_child_samples = trial.suggest_int('min_child_samples', 1, 100)
#     params = {
#         'device_type': 'gpu',
#         'boosting_type': 'gbdt',
#         'objective': 'multiclass',
#         'metric': 'multi_logloss',
#         'num_class': 7, 
#         'random_state': 0,
#         'verbose' : -1,
#         'lambda_l1' : lambda_l1,
#         'lambda_l2' : lambda_l2,
#         'num_leaves' : num_leaves,
#         'min_child_samples' : min_child_samples,
#     }

#     valid_scores = []
#     models = []
#     kf = StratifiedKFold(n_splits=3)
#     for fold, (train_indices, valid_indices) in enumerate(kf.split(X_train,y_train)):
#         X_cv_train, X_cv_valid = X_train.iloc[train_indices], X_train.iloc[valid_indices]
#         y_cv_train, y_cv_valid = y_train.iloc[train_indices], y_train.iloc[valid_indices]
#         lgb_train = lgb.Dataset(X_cv_train, y_cv_train)
#         lgb_val = lgb.Dataset(X_cv_valid, y_cv_valid)

#         model = lgb.train(
#                 params,
#                 lgb_train,
#                 valid_sets=[lgb_train, lgb_val],
#                 num_boost_round = 1000,
#                 early_stopping_rounds=100,
#                 verbose_eval=100)
#         y_valid_pred = model.predict(X_test)
#         score = accuracy_score(y_test, np.argmax(y_valid_pred, axis=1))
#         print(f'fold {fold} ACC: {score}')
#         valid_scores.append(score)

#         models.append(model)

#     cv_score = np.mean(valid_scores)
#     return cv_score

## 最適化の実行

In [ ]:
studyLGBM = optuna.create_study(direction='maximize')
studyLGBM.optimize(opt_LGBM, n_trials=15)
print(studyLGBM.best_params)
print(studyLGBM.best_value)
print(studyLGBM.best_trial)

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'num_class': 7, 
    'random_state': 0,
    'verbose' : -1,
    'lambda_l1' : studyLGBM.best_params["lambda_l1"],
    'lambda_l2' : studyLGBM.best_params["lambda_l2"],
    'num_leaves' : studyLGBM.best_params["num_leaves"],
    'min_child_samples' : studyLGBM.best_params["min_child_samples"],
}

lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_test, y_test, reference=lgb_train)

gbm = lgb.train(params,
                lgb_train,
                valid_sets=[lgb_train, lgb_val],
                num_boost_round = studyLGBM.best_params["num_boost_round"],
                early_stopping_rounds=100,
                verbose_eval=100)

In [ ]:
test_feature_df = test_df.copy()
test_feature_df = test_feature_df.drop("Id", axis=1)

In [ ]:
predicted = gbm.predict(test_feature_df)
pred_max = np.argmax(predicted, axis=1)

In [ ]:
sample_df.drop("Cover_Type", axis=1)
sample_df["Cover_Type"] = pred_max
sample_df["Cover_Type"] = sample_df["Cover_Type"] + 1

In [ ]:
sample_df.to_csv("submittion.csv", index=False)